<a href="https://colab.research.google.com/github/LinaTobon/Proyecticos-/blob/main/PersonalStalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Personal Stalker
En este proyecto aprenderemos a investigar y obtener los temas principales de los cuales publica una cuenta de Twitter que te interece.
De esta manera podrás tener un acercamiento a los aspectos que motivan a esta persona o empresa.

## Fases
1. Configuraciones, definiciones e importación de librerías
2. Declaración de funciones para:
   - Conexión a la API de twitter
   - Extracción de tweets de la cuenta elegida
   - Limpieza y transformación de datos.
5. Implementar modelo de tópicos
6. Visualización
7. Insights

### **1. Configuraciones, definiciones e importación de librerías**

In [ ]:
import warnings
warnings.filterwarnings("ignore") # Para ignoarar notificaciones molestas

# Instalación de paquetes
!pip install contractions
!pip install textsearch
!pip install tqdm
!pip install nltk
!pip install spacy

# importando librarias

import numpy as np
import pandas as pd
import collections
import itertools
import plotly.express as px
import matplotlib.pyplot as plt
import re
import contractions

import tweepy 

import spacy
from spacy.lang.en import English
#spacy.load('en')


import nltk
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')


from gensim import corpora
import pickle
import gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# ---- Settings ----#

#Twitter API credentials

consumer_key = "uP50Js2WwFNEllApoOS015PLb"
consumer_secret = "se7bTiiu24IwIXUg8qRhy4fXh6hPcsmgRMrvjj6kUfKP3tOHEG"
access_key = "963516449418563585-SYiUbAl4qfYiiaOUTmSJ0WX32GGj94J"
access_secret="IDoGQ3KeNjScSzk42Rt7HQTJMKazfm3CgEh8qFlJnTnNb"

# Definimos stop_words
stop_words = nltk.corpus.stopwords.words('english')
stop_words_spanish = nltk.corpus.stopwords.words('spanish')

# inicializamos stemmer y lemmatizer
ps = nltk.porter.PorterStemmer()
wnl = WordNetLemmatizer()
nlp = spacy.load("en_core_web_sm")

### **2. Declaración de funciones**

In [ ]:
#---- Conectandonos a la API de Twitter  y extrayendo datos ----#

def get_all_tweets(screen_name):

    """funcion que permite descargar los 3240 tweets mas recientes de la cuenta especificada"""
    
    # Inicializar tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    # Declarar una lista para almacenar losTweets
    alltweets = []  
    
    # Request para obtener los 200 tweets mas recientes
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    # Concatenando el resultado del request
    alltweets.extend(new_tweets)
    
    # obtenemos el id del tweet mas viejo que extrajimos 
    oldest = alltweets[-1].id - 1
    
    # Iteramos hasta que no haya mas tweets
    while len(new_tweets) > 0:
        
        # Request para obtener los tweets anteriores al oldest
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        alltweets.extend(new_tweets)
        
        # actualizamos el id del elemento mas antiguo
        oldest = alltweets[-1].id - 1
        
        print(f"...{len(alltweets)} tweets descargados")
    
    # Creamos un dataframe para almacenar los datos extraidos
    df_tweets = pd.DataFrame(columns=["id","created_at","text"])
    df_tweets["id"] =[tweet.id_str for tweet in alltweets]
    df_tweets["user"] =[tweet.user.id_str for tweet in alltweets]
    df_tweets["created_at"] =[tweet.created_at for tweet in alltweets]
    df_tweets["text"] =[tweet.text for tweet in alltweets]

    return df_tweets


#---- Limpieza de tweets ----#


def tag_wordnet(tagged_tokens):
    tag_map = {'j': wn.ADJ, 'v': wn.VERB, 'n': wn.NOUN, 'r': wn.ADV}
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wn.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens

def simple_text_preprocessor(document): 
    # lower case
    document = str(document).lower()
    
    # expand contractions
    document = contractions.fix(document)
    
    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)
    document = re.sub(r'http\S+', '',document)
    
    # simple porter stemming
    #document = ' '.join([ps.stem(word) for word in document.split()])
    tokens = nltk.word_tokenize(document)
    tagged_tokens = nltk.pos_tag(tokens)
    wordnet_tokens = tag_wordnet(tagged_tokens)
    lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in wordnet_tokens)
    lemmatized_tokens = nltk.word_tokenize(lemmatized_text)
    
    # stopwords removal
    document = ' '.join([token for token in lemmatized_tokens if token not in stop_words and token not in stop_words_spanish ])
    
    return document

def tokenize(text):
    lda_tokens = []
    parser = English()
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
historical_tweets = get_all_tweets('@Deloitte')
stp = np.vectorize(simple_text_preprocessor)
historical_tweets['clean_tweet'] = stp(historical_tweets['text'].values)


...400 tweets descargados
...600 tweets descargados
...800 tweets descargados
...1000 tweets descargados
...1200 tweets descargados
...1400 tweets descargados
...1600 tweets descargados
...1800 tweets descargados
...2000 tweets descargados
...2200 tweets descargados
...2400 tweets descargados
...2600 tweets descargados
...2800 tweets descargados
...3000 tweets descargados
...3200 tweets descargados
...3250 tweets descargados
...3250 tweets descargados


In [ ]:
historical_tweets

,id,created_at,text,user,clean_tweet
0,1605639241039486976,2022-12-21 19:00:01,We proudly introduce you three of our Deloitte...,8457092,proudly introduce three deloitte oyw ambassado...
1,1605593945152622592,2022-12-21 16:00:01,Want to join next year’s #OYW2023 cohort? Shar...,8457092,want join next year oyw cohort share u help re...
2,1605578841489133568,2022-12-21 15:00:00,See how Deloitte scored in @Gartner_inc. Crit...,8457092,see deloitte score gartner inc critical capabi...
3,1605563747094986752,2022-12-21 14:00:01,Deloitte scored highest in 2 Use Cases in 2022...,8457092,deloitte score high use case gartner inc criti...
4,1605548644006072321,2022-12-21 13:00:01,Approx. 17% of people globally live with a dis...,8457092,approx people globally live disability yet muc...
...,...,...,...,...,...
3245,1269316119971627008,2020-06-06 17:12:00,"From #technology and ethics to #leadership, #D...",8457092,technology ethic leadership deloittereview tac...
3246,1269282902342340613,2020-06-06 15:00:00,Did your organization adapt to the #blockchain...,8457092,organization adapt blockchain wave sign explor...
3247,1269041313028214786,2020-06-05 23:00:01,The new market landscape has challenged invest...,8457092,new market landscape challenge investment bank...
3248,1269002700064264195,2020-06-05 20:26:35,Earlier this week we expressed our stance agai...,8457092,earlier week express stance racism inequality ...


### **3. Implementar modelo de tópicos**

In [ ]:
text_data = [prepare_text_for_lda(i) for i in historical_tweets["clean_tweet"]]

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')


NUM_TOPICS = 4
ldamodel_1 = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel_1.save('model6.gensim')
topics = ldamodel_1.print_topics(num_words=5)

for topic in topics:
    print(topic)

(0, '0.014*"organization" + 0.014*"learn" + 0.012*"deloitte" + 0.012*"impact" + 0.011*"covid"')
(1, '0.022*"deloitte" + 0.009*"business" + 0.007*"leader" + 0.007*"challenge" + 0.007*"digital"')
(2, '0.031*"deloitte" + 0.023*"global" + 0.014*"explore" + 0.013*"leader" + 0.011*"industry"')
(3, '0.058*"deloitte" + 0.031*"service" + 0.029*"leader" + 0.017*"gartner" + 0.011*"recognize"')


## **4. Visualización**

In [ ]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=202731098cf65314064287c9d7d176b907e21638b5ec1d76677c6c9a78b1ba3d
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=754482e150cbbff6e2ed16878067993ae7aa1b2296b929aede78ad89b7eb80d4
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [ ]:
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import pickle
from gensim import corpora

#import pyLDAvis.gensim

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
#ldamodel_1= gensim.models.ldamodel.LdaModel.load('model6.gensim')

lda_display_1 = gensimvis.prepare(ldamodel_1, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display_1)
pyLDAvis.save_html(lda_display_1, 'TopicsUsersHistoricalTweets.html')

/usr/local/lib/python3.8/dist-packages/smart_open/smart_open_lib.py:496: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
